In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir(os.path.join(os.getcwd(), "..", "..", "data", "2017-18", "gws"));

In [3]:
teams = ["Arsenal", "Liverpool", "Chelsea", "Bournemouth", "Crystal Palace", "Manchester City", "Watford",
        "Manchester United", "Tottenham Hotspur", "Everton", "West Bromwich Albion", "Southampton",
        "Leicester City", "Newcastle United", "Brighton and Hove Albion", "Swansea City", "Stoke City",
        "Huddersfield Town", "West Ham United", "Burnley"]

In [4]:
teams.sort()

In [5]:
gws = np.arange(1, 39)

In [6]:
gws

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38])

In [7]:
df = pd.DataFrame();

In [8]:
for gw in gws:
    gw_df = pd.read_csv("gw{}.csv".format(gw), encoding = "ISO-8859-1")
    gw_df["GW"] = gw
    df = df.append(gw_df)

In [9]:
df["team"] = df["opponent_team"].apply(lambda team: teams[team-1])

In [10]:
df.columns

Index(['name', 'assists', 'attempted_passes', 'big_chances_created',
       'big_chances_missed', 'bonus', 'bps', 'clean_sheets',
       'clearances_blocks_interceptions', 'completed_passes', 'creativity',
       'dribbles', 'ea_index', 'element', 'errors_leading_to_goal',
       'errors_leading_to_goal_attempt', 'fixture', 'fouls', 'goals_conceded',
       'goals_scored', 'ict_index', 'id', 'influence', 'key_passes',
       'kickoff_time', 'kickoff_time_formatted', 'loaned_in', 'loaned_out',
       'minutes', 'offside', 'open_play_crosses', 'opponent_team', 'own_goals',
       'penalties_conceded', 'penalties_missed', 'penalties_saved',
       'recoveries', 'red_cards', 'round', 'saves', 'selected', 'tackled',
       'tackles', 'target_missed', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'winning_goals', 'yellow_cards', 'GW', 'team'],
      dtype='object')

In [11]:
player_mappings = {};

In [12]:
players = np.unique(df["name"])

In [13]:
players.sort()

In [14]:
len(players)

647

In [15]:
for index in range(len(players)):
    player = players[index]
    if (player not in player_mappings):
        player_mappings[player] = index + 1

In [16]:
player_names = []
player_ids = []

In [17]:
for name, id_ in player_mappings.items():
    player_names.append(name);
    player_ids.append(id_)

In [18]:
player_id_df = pd.DataFrame({"player_name": player_names, "player_id":player_ids })

In [19]:
player_id_df.to_csv("player_id_map.csv", index=False)

In [20]:
df["player_id"] = df["name"].apply(lambda name: player_id_df.loc[player_id_df['player_name'] == name, 'player_id'].iloc[0])

In [21]:
cols = ["player_id", "name", "GW", "influence", "creativity", "threat", "ict_index", "team", "opponent_team", 
        "was_home", "total_points"]

In [22]:
to_remove_cols = ["minutes", "goals_scored", "goals_conceded", "assists", "clean_sheets", "own_goals", "yellow_cards", "red_cards", "penalties_conceded", "penalties_saved", "penalties_missed",
       "saves", "dribbles", "attempted_passes", "bonus", "bps", "big_chances_created", "big_chances_missed",
       "clearances_blocks_interceptions", "completed_passes", "errors_leading_to_goal", "errors_leading_to_goal_attempt",
       "fouls", "key_passes", "offside", "open_play_crosses", "recoveries", "target_missed"]

In [23]:
cols = cols + to_remove_cols;

In [24]:
gwo = df[cols]

In [25]:
gwo.columns

Index(['player_id', 'name', 'GW', 'influence', 'creativity', 'threat',
       'ict_index', 'team', 'opponent_team', 'was_home', 'total_points',
       'minutes', 'goals_scored', 'goals_conceded', 'assists', 'clean_sheets',
       'own_goals', 'yellow_cards', 'red_cards', 'penalties_conceded',
       'penalties_saved', 'penalties_missed', 'saves', 'dribbles',
       'attempted_passes', 'bonus', 'bps', 'big_chances_created',
       'big_chances_missed', 'clearances_blocks_interceptions',
       'completed_passes', 'errors_leading_to_goal',
       'errors_leading_to_goal_attempt', 'fouls', 'key_passes', 'offside',
       'open_play_crosses', 'recoveries', 'target_missed'],
      dtype='object')

In [26]:
target_pgw = ["total_points", "ict_index", "influence", "creativity", "threat"];
target_pgw = target_pgw + to_remove_cols;
player_ids = np.unique(gwo["player_id"]);
new_df = pd.DataFrame(columns=list(gwo.columns));

In [27]:
for player_id in player_ids:
    df_ = gwo[gwo["player_id"] == player_id]
    df_.reset_index(inplace=True, drop=True)
    for index in range(len(df_)):
        instance = df_.iloc[index]
        gw = instance["GW"];
        for target in target_pgw:
            if (gw == 1 or index == 0):
                df_.ix[index, "{}_pgw".format(target)] = 0
            else:
                df_.ix[index, "{}_pgw".format(target)] = df_.iloc[index-1][target]
    new_df = new_df.append(df_)
    print("Done with player {}".format(player_id))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

Done with player 1
Done with player 2
Done with player 3
Done with player 4
Done with player 5
Done with player 6
Done with player 7
Done with player 8
Done with player 9
Done with player 10
Done with player 11
Done with player 12
Done with player 13
Done with player 14
Done with player 15
Done with player 16
Done with player 17
Done with player 18
Done with player 19
Done with player 20
Done with player 21
Done with player 22
Done with player 23
Done with player 24
Done with player 25
Done with player 26
Done with player 27
Done with player 28
Done with player 29
Done with player 30
Done with player 31
Done with player 32
Done with player 33
Done with player 34
Done with player 35
Done with player 36
Done with player 37
Done with player 38
Done with player 39
Done with player 40
Done with player 41
Done with player 42
Done with player 43
Done with player 44
Done with player 45
Done with player 46
Done with player 47
Done with player 48
Done with player 49
Done with player 50
Done with

Done with player 397
Done with player 398
Done with player 399
Done with player 400
Done with player 401
Done with player 402
Done with player 403
Done with player 404
Done with player 405
Done with player 406
Done with player 407
Done with player 408
Done with player 409
Done with player 410
Done with player 411
Done with player 412
Done with player 413
Done with player 414
Done with player 415
Done with player 416
Done with player 417
Done with player 418
Done with player 419
Done with player 420
Done with player 421
Done with player 422
Done with player 423
Done with player 424
Done with player 425
Done with player 426
Done with player 427
Done with player 428
Done with player 429
Done with player 430
Done with player 431
Done with player 432
Done with player 433
Done with player 434
Done with player 435
Done with player 436
Done with player 437
Done with player 438
Done with player 439
Done with player 440
Done with player 441
Done with player 442
Done with player 443
Done with pla

In [28]:
df_.reset_index(inplace=True, drop=True)

In [29]:
df_.drop(to_remove_cols, axis=1, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3930: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [30]:
df_["season"] = "2017-18"

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1]:
new_df

NameError: name 'new_df' is not defined

In [31]:
def getResult(row):
    if (row["team_h_score"] == row["team_a_score"]):
        return "D";
    elif (row["was_home"] is True and row["team_h_score"] < row["team_a_score"]):
        return "L";
    elif (row["was_home"] is True and row["team_h_score"] > row["team_a_score"]):
        return "W";
    elif (row["was_home"] is not True and row["team_h_score"] < row["team_a_score"]):
        return "W";
    elif (row["was_home"] is not True and row["team_h_score"] > row["team_a_score"]):
        return "L";

In [33]:
df_["result_pgw"] = df.apply(lambda row: getResult(row), axis=1)

ValueError: cannot reindex from a duplicate axis